In [1]:
import glob
import os
import pandas as pd
import numpy as np
import tqdm as tqdm
import matplotlib.pyplot as plt
import seaborn as sns

----

### Important

For each cancer type as well as the pan-cancer cohort we run dndscv with default options in both Hartwig and PCAWG samples:

1. To estimate global MHC-I (combining HLA-A, HLA-B and HLA-C) dn/ds ratios. Only in pancancer manner.
2. To estimate the gene-specific and pan-cancer dN/dS ratios in a pan-cancer manner for MHC-I genes. 
3. To identify positively selected genes by recurrent mutation of the list of 21 genes selected for anylsis in this publication (see Supp. Table 1). This analysis is performed in a pan-cancer and cancer type specific manner. 

The script ../scripts_positive_selection/mutations/run_driver_pipeline_dndscv.py was used to run the Hartwig and PCAWG a pan-cancer and cancer type specific analyses.
This scripts uses ../scripts_positive_selection/mutations/mutations/run_dndscv.R to compute the dndscv driver genes and dN/dS ratios

### Genes

In [2]:
# hla locus
mhc_I = ["HLA-A","HLA-B","HLA-C"]
# APP related proteins
transport_mhc = ["TAP1","TAP2","TAPBP"]
scaffold_mhc = ["B2M","CALR"]
#interferon
interferon=["JAK1","JAK2","STAT1","IRF2","APLNR","IFNGR1", "IFNGR2"]
#TFs HLA locus
tfs=["NLRC5","RFX5","CIITA"]
# CD58
cd58_nk = ["CD58"]
d_data = {"MHC-I":mhc_I,"transport_mhc":transport_mhc,"scaffold_mhc":scaffold_mhc,"IFN-gamma":interferon,"Activators-MHC":tfs,"cd58_nk":cd58_nk}
pallete = {"wt":"#ef8a62","alteration":"#67a9cf","alteration_primary":"#7570b3"}

In [3]:
total= []
for k in d_data:
    total+=d_data[k]

### Hartwig dataset

In [ ]:
output_hartwig="/home/fran/Documents/cuppen/HPC/tunnel/cuppen/projects/P0020_genetics_immune_escape/large_scale_primary_met/processed/hmf/positive_selection/dndscv/"

In [4]:
l,l1=[], []
for filein in glob.glob(f"{output_hartwig}/*.ci.tsv.gz"):
    ttype = os.path.basename(filein).split(".")[0]
    df = pd.read_csv(filein,sep="\t")
    df_d = pd.read_csv(filein.replace(".ci.",".results."),sep="\t")
    df=df[df["gene"].isin(total)]
    df_d = df_d[df_d["gene_name"].isin(total)]
    df["ttype"] = ttype
    df_d["ttype"] = ttype
    l.append(df)
    l1.append(df_d)
df_total = pd.concat(l)
df_total_d = pd.concat(l1)

In [5]:
df_total_d.sort_values("qglobal_cv").to_csv("../../results/data/dndscv_output_immune_genes_hmf.tsv.gz",sep="\t",index=False,compression="gzip")
df_total_d[df_total_d["qglobal_cv"]<.1].to_csv("../../results/data/MUT_positive_selection_HMF.tsv",sep="\t") # only significant

In [6]:
df_total_d[df_total_d["qglobal_cv"]<.1]

,gene_name,n_syn,n_mis,n_non,n_spl,n_ind,wmis_cv,wnon_cv,wspl_cv,wind_cv,pmis_cv,ptrunc_cv,pallsubs_cv,pind_cv,qmis_cv,qtrunc_cv,qallsubs_cv,pglobal_cv,qglobal_cv,ttype
13,CD58,0,3,0,0,1,336.748852,0.000000,0.000000,212.661070,0.000007,9.684713e-01,4.064594e-05,4.644831e-03,0.011633,9.955272e-01,4.891302e-02,3.111810e-06,4.465670e-03,Diffuse__large__B-cell__lymphoma
16,HLA-B,0,0,1,0,2,0.000000,1371.287781,1371.287781,161.044750,0.883371,9.770165e-04,4.190507e-03,3.767278e-04,0.971273,5.711779e-01,9.999474e-01,2.266815e-05,2.678975e-02,Diffuse__large__B-cell__lymphoma
12,B2M,1,1,0,0,3,11.911040,0.000000,0.000000,86.230721,0.079149,8.973482e-01,2.122826e-01,1.070398e-05,0.791533,9.637774e-01,9.860799e-01,3.179978e-05,4.914534e-02,Kidney__clear__cell__carcinoma
17,B2M,0,3,2,1,7,7.545842,82.950659,82.950659,35.350032,0.014809,1.141644e-05,7.829318e-06,1.606587e-07,0.875552,1.638341e-02,7.864941e-03,3.572487e-11,3.987491e-08,Colorectum__carcinoma
29,HLA-B,1,8,1,0,6,5.160554,11.138516,11.138516,13.132236,0.002332,8.950860e-02,3.687634e-03,1.474961e-04,0.690477,8.527777e-01,7.717527e-01,8.389552e-06,5.618483e-03,Colorectum__carcinoma
47,HLA-A,1,3,2,0,5,1.844576,20.150373,20.150373,10.454854,0.399184,5.453681e-03,1.829800e-02,9.488283e-04,0.907935,8.527777e-01,9.473870e-01,2.076671e-04,8.692167e-02,Colorectum__carcinoma
14,B2M,0,3,0,1,3,10.066629,47.654570,47.654570,45.445549,0.008991,1.889372e-02,3.372658e-03,6.893604e-05,0.845466,8.710051e-01,8.365440e-01,3.783756e-06,5.067963e-03,Skin__melanoma
28,B2M,2,15,9,4,16,3.731710,33.698190,33.698190,18.354051,0.000652,4.540812e-14,6.661338e-14,1.073650e-11,0.147281,4.561473e-11,4.317192e-11,0.000000e+00,0.000000e+00,pancancer
48,HLA-A,11,24,15,2,14,1.462092,10.899306,10.899306,5.920520,0.198323,9.631451e-11,7.114442e-10,2.431934e-05,0.883074,7.154733e-08,3.324099e-07,5.655476e-13,2.318861e-10,pancancer
64,HLA-B,12,26,7,2,19,1.548579,5.937452,5.937452,8.133033,0.126463,1.069171e-04,4.471238e-04,7.374858e-08,0.840779,3.521427e-02,7.583199e-02,8.288294e-10,2.561848e-07,pancancer


### PCAWG dataset

In [ ]:
output_pcawg="/home/fran/Documents/cuppen/HPC/tunnel/cuppen/projects/P0020_genetics_immune_escape/large_scale_primary_met/processed/pcawg/positive_selection/dndscv/"

In [9]:
l,l1=[], []
for filein in glob.glob(f"{output_pcawg}/*.ci.tsv.gz"):
    ttype = os.path.basename(filein).split(".")[0]
    df = pd.read_csv(filein,sep="\t")
    df_d = pd.read_csv(filein.replace(".ci.",".results."),sep="\t")
    df=df[df["gene"].isin(total)]
    df_d = df_d[df_d["gene_name"].isin(total)]
    df["ttype"] = ttype
    df_d["ttype"] = ttype
    l.append(df)
    l1.append(df_d)
df_total = pd.concat(l)
df_total_d = pd.concat(l1)

In [10]:
df_total_d.sort_values("qglobal_cv").to_csv("../results/data/dndscv_output_immune_genes_pcawg.tsv.gz",sep="\t",index=False,compression="gzip")
df_total_d[df_total_d["qglobal_cv"]<.1].to_csv("../results/data/MUT_positive_selection_PCAWG.tsv",sep="\t") # only significant

In [11]:
df_total_d[df_total_d["qglobal_cv"]<.1]

,gene_name,n_syn,n_mis,n_non,n_spl,n_ind,wmis_cv,wnon_cv,wspl_cv,wind_cv,pmis_cv,ptrunc_cv,pallsubs_cv,pind_cv,qmis_cv,qtrunc_cv,qallsubs_cv,pglobal_cv,qglobal_cv,ttype
3,B2M,0,11,5,1,5,61.618363,371.500360,371.500360,108.028419,1.711530e-09,1.143907e-11,1.509903e-14,1.029211e-05,4.912334e-06,7.660746e-08,7.583867e-11,0.000000e+00,0.000000e+00,Diffuse__large__B-cell__lymphoma
13,CD58,0,0,2,2,1,0.000000,258.961580,258.961580,65.682088,5.936073e-01,3.161678e-08,1.188904e-07,1.463931e-02,8.036403e-01,9.074468e-05,1.492892e-04,3.684426e-08,5.287414e-05,Diffuse__large__B-cell__lymphoma
18,HLA-B,0,5,2,1,1,22.475429,177.715000,177.715000,31.913489,1.337234e-04,2.174982e-06,4.686941e-07,2.896927e-02,7.462878e-02,3.972507e-03,4.708266e-04,2.595360e-07,2.744388e-04,Diffuse__large__B-cell__lymphoma
9,B2M,1,17,6,1,7,14.978107,65.820672,65.820672,33.929853,2.652700e-11,1.520341e-10,0.000000e+00,2.535773e-08,4.099646e-08,2.349628e-07,0.000000e+00,0.000000e+00,0.000000e+00,pancancer
28,HLA-A,3,5,7,2,5,1.403459,25.353874,25.353874,9.378269,5.336295e-01,1.989837e-09,1.429466e-08,5.923218e-04,9.736949e-01,2.498613e-06,1.025693e-05,2.243328e-10,1.554162e-07,pancancer
35,HLA-B,1,13,2,2,7,4.697558,15.264558,15.264558,13.513120,2.746188e-04,2.906790e-04,1.105880e-05,9.405607e-06,1.103473e-01,1.358147e-01,4.534335e-03,2.494949e-09,1.392389e-06,pancancer
68,CD58,0,3,3,2,3,1.301947,18.968152,18.968152,9.086144,6.981241e-01,2.321816e-05,1.261295e-04,6.334899e-03,9.839546e-01,1.504761e-02,4.445735e-02,1.201714e-05,3.499077e-03,pancancer
